In [9]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [10]:
# Loadthe trained model, scaler pickle, onehot
model=load_model('model.h5')

#load the encoder and scaler
with open('label_encoder_Gender.pkl','rb') as file:
    label_encoder_Gender=pickle.load(file)
with open('onehot_encoder_geography.pkl','rb') as file:
    label_encoder_geography=pickle.load(file)
with open('scaler.pkl','rb') as file:
    scaler=pickle.load(file)

In [11]:
# Example input data
input_data={
    'CreditScore':608,
    'Geography':'Spain',
    'Gender':'Female',
    'Age':41,
    'Tenure':1,
    'Balance':83807,
    'NumOfProducts':1,
    'HasCrCard':0,
    'IsActiveMember':1,
    'EstimatedSalary':112542
}

In [12]:
# one-hot encode 'Geography'
geo_encoded=label_encoder_geography.transform([[input_data['Geography']]]).toarray()
geo_encoded_df=pd.DataFrame(geo_encoded, columns=label_encoder_geography.get_feature_names_out(['Geography']))
geo_encoded_df

c:\Users\DELL\OneDrive\app\Desktop\ANN_MODEL\venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,0.0,0.0,1.0


In [13]:
input_df=pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,608,Spain,Female,41,1,83807,1,0,1,112542


In [14]:
input_df['Gender']=label_encoder_Gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,608,Spain,0,41,1,83807,1,0,1,112542


In [15]:
# concatination one hot encoded 
input_df=pd.concat([input_df.drop("Geography",axis=1),geo_encoded_df],axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,608,0,41,1,83807,1,0,1,112542,0.0,0.0,1.0


In [16]:
# Scaling the input data
input_scaled=scaler.transform(input_df)
input_scaled

array([[-0.38963278, -1.09997489,  0.1912806 , -1.38035469,  0.09314609,
        -0.89163935, -1.57810057,  0.95214374,  0.24045933, -1.02020406,
        -0.57581067,  1.77464858]])

In [17]:
# predict churn
prediction=model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 337ms/step


array([[0.2519044]], dtype=float32)

In [18]:
prediction_proba=prediction[0][0]


In [19]:
if prediction_proba > 0.5:
    print('The customer is likey to churn')
else:
    print('The customer is not likely to churn')

The customer is not likely to churn
